In [1]:
import json
import re
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
import pandas as pd


# 去除标点符号的函数
def remove_punctuation(text):
    """
    移除输入文本中的标点符号。
    Args:
    - text: 输入的文本字符串。

    Returns:
    - 去除标点符号后的文本字符串。
    """
    return re.sub(r'[^\w\s]', '', text)


# 最长公共子文本串算法
def algrot(text1, text2):
    """
    使用动态规划算法，计算两个字符串的最长公共子文本串。
    Args:
    - text1: 第一个字符串。
    - text2: 第二个字符串。

    Returns:
    - 最长公共子文本串。
    """
    m = len(text1)
    n = len(text2)
    max_len = 0
    ending_index = m
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # 动态规划填充表格，计算公共子串长度
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_len:
                    max_len = dp[i][j]
                    ending_index = i
            else:
                dp[i][j] = 0

    # 返回最长公共子串
    return text1[ending_index - max_len: ending_index]


# 查找文本中的所有最长公共子串
def find_all_common_substrings(text1, text2):
    """
    查找文本1和文本2的所有公共子串，返回一个子串列表。
    Args:
    - text1: 第一个字符串。
    - text2: 第二个字符串。

    Returns:
    - 所有公共子串的列表。
    """
    m = len(text1)
    n = len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    substrings = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                substrings.append(text1[i - dp[i][j]:i])
            else:
                dp[i][j] = 0

    return list(set(substrings))  # 去重


# 计算匹配占比
def calculate_match_ratio(substring, content):
    """
    计算公共子串在目标文本中的占比。
    Args:
    - substring: 公共子串。
    - content: 目标文本。

    Returns:
    - 公共子串长度与目标文本长度的比值。
    """
    return len(substring) / len(content) if content else 0


# 查找最佳匹配
def find_best_match(predict_item, labels):
    """
    在 label 列表中为 predict_item 查找最佳匹配的 label 项。
    Args:
    - predict_item: 需要匹配的 predict 项。
    - labels: label 列表。

    Returns:
    - 最佳匹配的 label 项。
    """
    best_match = None
    highest_ratio = 0

    # 去除标点符号
    predict_content_no_punc = remove_punctuation(predict_item['content'])

    for label in labels:
        label_content_no_punc = remove_punctuation(label['content'])

        # 找到所有公共子串
        common_substrings = find_all_common_substrings(predict_content_no_punc, label_content_no_punc)

        # 过滤公共子串，确保其长度占 predict_content 的70%以上
        valid_substrings = [sub for sub in common_substrings if len(sub) / len(predict_content_no_punc) >= 0.8]

        # 如果没有符合的子串，跳过这次匹配
        if not valid_substrings:
            continue

        # 计算公共子串与目标文本的占比，找出占比最高的
        for substring in valid_substrings:
            ratio = calculate_match_ratio(substring, label_content_no_punc)
            if ratio > highest_ratio:
                highest_ratio = ratio
                best_match = label
    if best_match is None:
        return {"content": "其它", "type": "其它"}

    return best_match


# 处理预测与匹配
def process_predictions(predict, labels):
    """
    处理 predict 列表，找到每个 predict 项在 label 列表中的最佳匹配项。
    Args:
    - predict: 需要处理的 predict 列表。
    - labels: label 列表。

    Returns:
    - 包含匹配结果的 predict 列表。
    """
    results = []

    # 遍历 predict 列表中的所有项
    for predict_item in predict:
        best_match = find_best_match(predict_item, labels)

        # 只要找到匹配的 label 项，不管 type 是否匹配，都进行记录
        if best_match:
            predict_item['matched_content'] = best_match['content']
            predict_item['matched_type'] = best_match['type']

        results.append(predict_item)

    return results


# 提取 filtered_result 中 matched_type 为 "发起" 和 "评价" 的数据
def filter_results(results):
    """
    从 results 中提取 matched_type 为 '发起' 和 '评价' 的项，构造 filtered_result。
    Args:
    - results: 原始匹配结果列表

    Returns:
    - filtered_result: 过滤后的列表，仅包含 '发起' 和 '评价' 类型的项。
    """
    filtered_result = [item for item in results if item['matched_type'] in ['发起', '评价']]
    return filtered_result


# 基于 filtered_result 计算 Precision, Recall, F1, 和 Accuracy
def calculate_metrics(filtered_result):
    """
    计算 '发起' 和 '评价' 两个类别的 Precision, Recall, F1 Score 和 Accuracy。
    Args:
    - filtered_result: 经过过滤的匹配结果列表

    Returns:
    - metrics: 包含 precision, recall, f1 和 accuracy 的字典
    """
    # 提取实际标签（true labels）和预测标签（predicted labels）
    y_true = [item['type'] for item in filtered_result]  # 实际标签
    y_pred = [item['matched_type'] for item in filtered_result]  # 预测标签

    # 计算 precision, recall, f1 和 accuracy
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    accuracy = accuracy_score(y_true, y_pred)

    # 返回计算结果
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    }

    # 定义 label 和 predict 列表


label = [
    {
        "type": "评价",
        "content": "不行，你按照老师的要求去做。"
    },
    {
        "type": "其它",
        "content": "请坐。"
    },
    {
        "type": "评价",
        "content": "要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。"
    },
    {
        "type": "发起",
        "content": "好，往下面来。你是读喜鹊的话啊。喜鹊来啊，喜鹊小喜鹊停来上来来表演啊。忘记了，可以稍微看一下啊。"
    },
    {
        "type": "评价",
        "content": "杏四桃三。请坐啊。虽然她讲的不流畅啊，但是他确实是勇敢的上台。而且他刚刚坐在下面是认真听的啊。最起码比下面不敢举手的小朋友要勇敢。这一点我就要鼓励。"
    },
    {
        "type": "发起",
        "content": "你是喜鹊来。再请只喜鹊。"
    },
    {
        "type": "评价",
        "content": "喜鹊没有刚刚乌鸦讲的好。"
    },
    {
        "type": "发起",
        "content": "对，我要请只非常棒的小喜鹊。嗯，哎，掌声鼓励。"
    },
    {
        "type": "讲解",
        "content": "但是讲小故事的小朋友不是背书，可以加上动作。像刚刚几只小乌鸦就扮演的非常好啊。"
    },
    {
        "type": "发起",
        "content": "好，接下来请来演小杜鹃。好的，你来。嗯，我的书在上面。你实在忘记了，可以看一下提示啊。"
    },
    {
        "type": "评价",
        "content": "你有这个耐心吗？你前面用过了，前面乌鸦已经说过了。请坐好，掌声鼓励一下。小杜鹃有点严肃啊。好啦，这小杜鹃可爱一点，活泼一点。猴哥猴哥，着急一点好不好？"
    },
    {
        "type": "发起",
        "content": "来，谁再来演一演？你来罗若涵。你有这个耐心，谁说过了啊。"
    },
    {
        "type": "发起",
        "content": "你有这个耐心，谁说过了啊。"
    }
]
predict = [

    {
        "type": "发起",
        "content": "对，"
    },
    {
        "type": "发起",
        "content": "没有举手的我也要请。"
    },
    {
        "type": "其它",
        "content": "上课，"
    },
    {
        "type": "讲解",
        "content": "就要积极开动脑筋。"
    },
    {
        "type": "讲解",
        "content": "像上次开汇总课一样。"
    },
    {
        "type": "讲解",
        "content": "每个同学都认真举手。"
    },
    {
        "type": "发起",
        "content": "好，往下面来。"
    },
    {
        "type": "发起",
        "content": "你是读喜鹊的话啊。"
    },
    {
        "type": "发起",
        "content": "喜鹊来啊，喜鹊小喜鹊停来上来来表演啊。"
    },
    {
        "type": "发起",
        "content": "忘记了，可以稍微看一下啊。"
    },
    {
        "type": "讲解",
        "content": "杏四桃三。"
    },
    {
        "type": "其它",
        "content": "请坐啊。"
    },
    {
        "type": "评价",
        "content": "虽然她讲的不流畅啊，"
    },
    {
        "type": "讲解",
        "content": "但是他确实是勇敢的上台。"
    },
    {
        "type": "讲解",
        "content": "而且他刚刚坐在下面是认真听的啊。"
    },
    {
        "type": "讲解",
        "content": "最起码比下面不敢举手的小朋友要勇敢。"
    },
    {
        "type": "评价",
        "content": "这一点我就要鼓励。"
    },
    {
        "type": "发起",
        "content": "你是喜鹊来。"
    },
    {
        "type": "发起",
        "content": "再请只喜鹊。"
    },
    {
        "type": "讲解",
        "content": "喜鹊没有刚刚乌鸦讲的好。"
    },
    {
        "type": "发起",
        "content": "对，我要请只非常棒的小喜鹊。"
    },
    {
        "type": "讲解",
        "content": "嗯，哎，掌声鼓励。"
    },
    {
        "type": "讲解",
        "content": "但是讲小故事的小朋友不是背书，可以加上动作。"
    },
    {
        "type": "讲解",
        "content": "像刚刚几只小乌鸦就扮演的非常好啊。"
    },
    {
        "type": "发起",
        "content": "好，接下来请来演小杜鹃。"
    },
    {
        "type": "发起",
        "content": "好的，你来。"
    },
    {
        "type": "讲解",
        "content": "嗯，我的书在上面。"
    },
    {
        "type": "发起",
        "content": "你实在忘记了，可以看一下提示啊。"
    },
    {
        "type": "讲解",
        "content": "你有这个耐心吗？"
    },
    {
        "type": "讲解",
        "content": "你前面用过了，前面乌鸦已经说过了。"
    },
    {
        "type": "其它",
        "content": "请坐好，"
    },
    {
        "type": "讲解",
        "content": "掌声鼓励一下。"
    },
    {
        "type": "讲解",
        "content": "小杜鹃有点严肃啊。"
    },
    {
        "type": "发起",
        "content": "好啦，这小杜鹃可爱一点，活泼一点。"
    },
    {
        "type": "发起",
        "content": "猴哥猴哥，着急一点好不好？"
    },
    {
        "type": "发起",
        "content": "来，谁再来演一演？"
    },
    {
        "type": "发起",
        "content": "你来罗若涵。"
    },
    {
        "type": "讲解",
        "content": "结果你有这么大耐心吗？"
    },
    {
        "type": "讲解",
        "content": "你有这个耐心，"
    },
    {
        "type": "讲解",
        "content": "谁说过了啊。"
    }
]
results = process_predictions(predict, label)
print(f"results_no_filter: {results}")
# 提取过滤结果
filtered_result = filter_results(results)
print("filtered_result", filtered_result)

In [2]:
    # 执行更新后的处理


results_no_filter: [{'type': '其它', 'content': '不行，', 'matched_content': '不行，你按照老师的要求去做。', 'matched_type': '评价'}, {'type': '讲解', 'content': '你按照老师的要求去做。', 'matched_content': '不行，你按照老师的要求去做。', 'matched_type': '评价'}, {'type': '其它', 'content': '请坐。', 'matched_content': '请坐。', 'matched_type': '其它'}, {'type': '评价', 'content': '要批评。', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。', 'matched_type': '评价'}, {'type': '讲解', 'content': '你等会儿三十秒的话都到我这来。', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。', 'matched_type': '评价'}, {'type': '讲解', 'content': '一定要积极思考，', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是在那里随便想讲话就讲话。你坐端正，坐好，快点坐下来。对，没有举手的我也要请。上课，就要积极开动脑筋。像上次开汇总课一样。每个同学都认真举手。', 'matched_type': '评价'}, {'type': '讲解', 'content': '不是自己坐在那里。', 'matched_content': '要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读

In [3]:
df = pd.DataFrame(filtered_result)
df

,type,content,matched_content,matched_type
0,其它,不行，,不行，你按照老师的要求去做。,评价
1,讲解,你按照老师的要求去做。,不行，你按照老师的要求去做。,评价
2,评价,要批评。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
3,讲解,你等会儿三十秒的话都到我这来。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
4,讲解,一定要积极思考，,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
5,讲解,不是自己坐在那里。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
6,讲解,想不读就不读。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
7,讲解,也不是在那里随便想讲话就讲话。,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
8,其它,你坐端正，,要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是自己坐在那里。想不读就不读。也不是...,评价
9,其它,坐好，,你有这个耐心吗？你前面用过了，前面乌鸦已经说过了。请坐好，掌声鼓励一下。小杜鹃有点严肃啊。好...,评价


In [65]:
def remove_punctuation(text):
    """
    移除输入文本中的标点符号。
    Args:
    - text: 输入的文本字符串。

    Returns:
    - 去除标点符号后的文本字符串。
    """
    return re.sub(r'[^\w\s]', '', text)


# 查找文本中的所有最长公共子串
def find_all_common_substrings(text1, text2):
    """
    查找文本1和文本2的所有公共子串，返回一个子串列表。
    Args:
    - text1: 第一个字符串。
    - text2: 第二个字符串。

    Returns:
    - 所有公共子串的列表。
    """
    m = len(text1)
    n = len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    substrings = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                substrings.append(text1[i - dp[i][j]:i])
            else:
                dp[i][j] = 0

    return list(set(substrings))  # 去重


def find_best_match_expanded_with_results(predict_items, labels):
    """
    在 label 列表中为多个 predict_item 查找最佳匹配的 label 项，并将匹配结果保存到 predict_items 中。
    返回包含匹配结果的 predict_items 列表。
    """
    start_index = 0  # 初始化从第一个 label 开始

    for predict_item in predict_items:
        best_match = None
        highest_ratio = 0
        best_match_index = -1  # 用于记录匹配的索引
        
        # 去除标点符号
        predict_content_no_punc = remove_punctuation(predict_item['content'])

        # 只在 start_index 到 start_index + 5 的范围内进行匹配
        for i in range(start_index, min(start_index + 5, len(labels))):
            label = labels[i]
            label_content_no_punc = remove_punctuation(label['content'])

            # 找到所有公共子串
            common_substrings = find_all_common_substrings(predict_content_no_punc, label_content_no_punc)

            # 放宽公共子串长度的限制，确保匹配的更宽松
            valid_substrings = [sub for sub in common_substrings if len(sub) / len(predict_content_no_punc) >= 0.3]

            # 如果没有符合的子串，跳过这次匹配
            if not valid_substrings:
                continue

            # 计算公共子串与目标文本的占比，找出占比最高的
            for substring in valid_substrings:
                ratio = calculate_match_ratio(substring, label_content_no_punc)
                if ratio > highest_ratio:
                    highest_ratio = ratio
                    best_match = label
                    best_match_index = i  # 记录匹配成功的索引

        if best_match is None:
            predict_item['best_match'] = {"content": "其它", "type": "其它"}
        else:
            predict_item['best_match'] = best_match

            # 如果匹配的索引更新了，才更新 start_index
            if best_match_index != -1 and best_match_index > start_index:
                start_index = best_match_index  # 更新起始索引为匹配成功的索引

    return predict_items



In [66]:
label = [
{"type": "发起", "content": "请袁艺喊“开始上课”，声音要响亮啊。"},
{"type": "其它", "content": "同学们好，请坐。"},
{"type": "评价", "content": "真棒。"},
{"type": "讲解", "content": "昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。"},
{"type": "发起", "content": "我们来跟他打打招呼。“猴子”。"},
{"type": "讲解", "content": "昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？"},
{"type": "发起", "content": "好，现在用眼睛看，用心记住这个字，“猴子”。"},
{"type": "讲解", "content": "哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。"},
{"type": "发起", "content": "它分别种了什么树呢？谁来说说？于凯，你来说说看。"},
{"type": "其它", "content": "你慢讲啊。嗯，"},
{"type": "发起", "content": "然后呢？"},
{"type": "讲解", "content": "然后种了杏树。"},
{"type": "发起", "content": "最后呢？"},
{"type": "讲解", "content": "最后还种了桃树。"},
{"type": "发起", "content": "到最后怎么样啊？"},
{"type": "发起", "content": "好，于凯，你能不能连起来把这句话来说一说？“猴子种了”。"},
{"type": "评价", "content": "非常棒，请坐。"},
{"type": "发起", "content": "我们像他一样，用一句话把这个故事，哎，就讲出来了。来，我们一起来试试看。“猴子种了”。"},
{"type": "评价", "content": "瞧，咱们一句话就能把故事的意思讲出来了。小朋友们真能干。"},
{"type": "发起", "content": "好，那我们来看看哦，他为什么什么都没种成呢？我们来读课文的第一段。来，谁来读？"},
{"type": "其它", "content": "那等一下啊。"},
{"type": "发起", "content": "好，你来读。“猴子”。"},
{"type": "其它", "content": "嗯嗯，你先不要坐下，"},
{"type": "发起", "content": "这个“天天”说明什么呀？"}
]

predict = [
  {"type": "发起", "content": "请袁艺喊“开始上课”，声音要响亮啊。"},
  {"type": "其它", "content": "同学们好，请坐。"},
  {"type": "评价", "content": "真棒。"},
  {"type": "讲解", "content": "昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。"},
  {"type": "发起", "content": "哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。它分别种了什么树呢？谁来说说？"},
  {"type": "发起", "content": "于凯，你来说说看。"},
  {"type": "其它", "content": "你慢讲啊。"},
  {"type": "发起", "content": "嗯，然后呢？"},
  {"type": "讲解", "content": "然后种了杏树。"},
  {"type": "发起", "content": "最后呢？"},
  {"type": "讲解", "content": "最后还种了桃树。"},
  {"type": "发起", "content": "到最后怎么样啊？"},
  {"type": "发起", "content": "好，于凯，你能不能连起来把这句话来说一说？“猴子种了”。"},
  {"type": "评价", "content": "非常棒，请坐。"},
  {"type": "讲解", "content": "我们像他一样，用一句话把这个故事，哎，就讲出来了。来，我们一起来试试看。“猴子种了”。瞧，咱们一句话就能把故事的意思讲出来了。小朋友们真能干。"},
  {"type": "讲解", "content": "好，那我们来看看哦，他为什么什么都没种成呢？我们来读课文的第一段。来，谁来读？那等一下啊。"},
  {"type": "发起", "content": "好，你来读。“猴子”。"},
  {"type": "其它", "content": "嗯嗯，你先不要坐下，这个“天天”说明什么呀？"}
]


In [68]:
find_best_match_expanded_with_results(predict, label)

[{'type': '发起',
  'content': '请袁艺喊“开始上课”，声音要响亮啊。',
  'best_match': {'type': '发起', 'content': '请袁艺喊“开始上课”，声音要响亮啊。'}},
 {'type': '其它',
  'content': '同学们好，请坐。',
  'best_match': {'type': '其它', 'content': '同学们好，请坐。'}},
 {'type': '评价',
  'content': '真棒。',
  'best_match': {'type': '评价', 'content': '真棒。'}},
 {'type': '讲解',
  'content': '昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。',
  'best_match': {'type': '讲解',
   'content': '昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？'}},
 {'type': '发起',
  'content': '哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。它分别种了什么树呢？谁来说说？',
  'best_match': {'type': '讲解', 'content': '哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。'}},
 {'type': '发起',
  'content': '于凯，你来说说看。',
  'best_match': {'type': '发起', 'content': '它分别种了什么树呢？谁来说说？于凯，你来说说看。'}},
 {'type': '其它',
  'content': '你慢讲啊。',
  'best_match': {'type': '其它', 'content': '你慢讲啊。嗯，'}},
 {'type':

In [20]:

best_match = find_best_match(predict_item, label)

predict_item: {'type': '发起', 'content': '对，我要请只非常棒的小喜鹊。嗯，哎，掌声鼓励。'}
predict_item: {'type': '发起', 'content': '要批评。你等会儿三十秒的话都到我这来。'}
predict_item: {'type': '发起', 'content': '想不读就不读。也不是在那里随便想讲话就讲话。'}
predict_item: {'type': '发起', 'content': '对，'}


In [ ]:
[
  {"type":"其它", "content":"乌鸦说过了，咱们不重复。你不着急吗？意思就是说你会着急的，是不是啊？意思一样啊。"},
  {"type":"评价", "content":"好，很好，掌声鼓励。"},
  {"type":"讲解", "content":"听了三只鸟儿的话，猴子可有想法了。猴子是怎么想的？听了乌鸦的话，猴子一想，你们瞧，可等不及，也等不及，还是等不及。说明这只猴子越来越没有耐心。他越来越没有耐心了，是吧？"},
  {"type":"发起", "content":"现在呀我们拿起书本，把这个故事啊从头到尾再来背一背。用上刚刚老师教你们的，咱们去把故事啊讲流利讲通顺，然后讲的有感情啊。"},
  {"type":"发起", "content":"猴子种了一棵梨树苗，预备起。"},
  {"type":"其它", "content":"猴子，这些东西放下，我们来看一下板书。猴子呀他一开始很勤劳啊，还是种了一棵梨树。"},
  {"type":"讲解","content":"可是当乌鸦来劝说它的时候，它就拔掉了梨树，改种了。正当杏树成活的时候，喜鹊来劝他了。于是猴子又拔掉了，改种了桃树。正当桃树成活的时候，杜鹃又来劝说它了。这时候猴子又拔掉了，改种了。可是有一句农谚叫，樱桃好吃，树难栽，一连几年都没有栽活。到最后这只猴子什么树也没有成。"},
  {"type":"讲解", "content":"对呀，这只猴子什么树也没有种成。除了它没耐心，它还没什么？"},
  {"type":"评价", "content":"你说，说的真好。"},
  {"type":"讲解", "content":"还没有主见，别人说什么他就听什么，没有自己的想法，是不是啊？到最后什么都没有啊。"},
  {"type":"发起", "content":"那么这只小猴呢，也挺可怜的。你想对他说什么？"}
]

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("data/726四分类法.xlsx")

In [3]:
df

,原始文本,text,label,gpt4o_predict,glm4_predict,qwen_predict,qwen_72B_predict,qwen_max_predict
0,请袁艺喊开始上课， 声音要响亮啊，同学们好，请坐真棒， 昨天呀，我们一起学习了课文八。 今天...,请袁艺喊“开始上课”，声音要响亮啊。同学们好，请坐。真棒。昨天呀，我们一起学习了课文八。今天...,"[\n{""type"": ""发起"", ""content"": ""请袁艺喊“开始上课”，声音要响亮...","{\n ""result"": [\n {""type"": ""发起"", ""content""...","[\n {""type"": ""发起"", ""content"": ""请袁艺喊“开始上课”，声音要...","[\n {""type"":""其它"",""content"":""请袁艺喊‘开始上课’，声音要响亮啊...","\n[\n{""type"":""发起"",""content"":""请袁艺喊‘开始上课’，声音要响亮啊...","\n[\n {""type"": ""发起"", ""content"": ""请袁艺喊“开始上课”，声..."
1,嗯，细心，还有呢，他每天都去浇，天天都去浇水施肥，说明他怎么样，对，非常勤劳，猴子是不是很勤...,嗯，细心。还有呢？他每天都去浇，天天都去浇水施肥。说明他怎么样？对，非常勤劳。猴子是不是很勤...,"[\n{""type"": ""评价"", ""content"": ""嗯，细心。""},\n{""type...","{\n ""result"": [\n {""type"": ""评价"", ""content""...","[\n {""type"": ""评价"", ""content"": ""嗯，细心。""},\n {""...","[\n {""type"":""其它"", ""content"":""嗯，细心。""},\n {""ty...","[\n {""type"": ""其它"", ""content"": ""嗯，细心。""},\n {""...","\n[\n {""type"": ""评价"", ""content"": ""嗯，细心。""},\n ..."
2,好的，你连叫两声， 猴哥给我听听，哎，猴哥听见了来，我们学着他的样子， 小乌鸦们叫两声猴哥，...,好的，你连叫两声。猴哥，给我听听。哎，猴哥听见了。来，我们学着他的样子。小乌鸦们，叫两声猴哥...,"[\n{\n""type"":""发起"",\n""content"":""好的，你连叫两声。猴哥，给我听...","{\n ""result"": [\n {""type"": ""发起"", ""content""...","[\n {""type"": ""发起"", ""content"": ""好的，你连叫两声。猴哥，给我...","[\n {""type"":""发起"",""content"":""好的，你连叫两声。猴哥，给我听听。...","[\n{""type"":""发起"",""content"":""好的，你连叫两声。""},\n{""typ...","\n[\n {""type"": ""其它"", ""content"": ""好的，你连叫两声。""},..."
3,哪知小乌鸦再来试试看，再来劝劝猴哥，你来，欢迎陈一坤，哎，这只小乌鸦讲的好不好？非常棒， ...,哪知小乌鸦再来试试看，再来劝劝猴哥。你来，欢迎陈一坤。哎，这只小乌鸦讲的好不好？非常棒，真好...,"[\n{""type"": ""发起"", ""content"": ""哪知小乌鸦再来试试看，再来劝劝猴...","{\n ""result"": [\n {""type"": ""讲解"", ""content""...","[\n {""type"": ""发起"", ""content"": ""哪知小乌鸦再来试试看，再来劝...","[\n {""type"":""发起"",""content"":""哪知小乌鸦再来试试看，再来劝劝猴哥...","[\n {""type"": ""发起"", ""content"": ""哪知小乌鸦再来试试看，再来劝...","\n[\n {""type"": ""发起"", ""content"": ""你来，欢迎陈一坤。""},..."
4,真的是男孩子今天读的真好， 而且很多女孩子也在认真听，我也要表扬， 非常好啊，来同学们，你们...,真的是男孩子今天读的真好，而且很多女孩子也在认真听，我也要表扬，非常好啊。来，同学们，你们来...,"[\n {""type"": ""评价"", ""content"": ""真的是男孩子今天读的真好...","{\n ""result"": [\n {""type"": ""评价"", ""content""...","[\n {""type"": ""评价"", ""content"": ""真的是男孩子今天读的真好，而...","[\n {""type"":""评价"",""content"":""真的是男孩子今天读的真好，而且很多...","[\n{""type"":""评价"",""content"":""真的是男孩子今天读的真好，而且很多女孩...","\n[\n {""type"": ""评价"", ""content"": ""真的是男孩子今天读的真好..."
5,好，把三句话来说一说， 我再问乌鸦对猴子说的最后一句话是什么，好喜鹊对猴子说的最后一句话， ...,好，把三句话来说一说。我再问乌鸦对猴子说的最后一句话是什么。好，喜鹊对猴子说的最后一句话。杜...,"[\n {""type"":""发起"",""content"":""好，把三句话来说一说。""},\...","{\n ""result"": [\n {""type"": ""发起"", ""content""...","[\n {""type"": ""发起"", ""content"": ""好，把三句话来说一说。""},...","[\n {""type"":""发起"",""content"":""好，把三句话来说一说。""},\n ...","[\n{""type"":""发起"",""content"":""好，把三句话来说一说。""},\n{""t...","\n[\n {""type"": ""发起"", ""content"": ""好，把三句话来说一说。""..."
6,同桌如果讲喜趣的话呢， 那要反过来讲， 等一会下课再出去，啊， 是不是讲得差不多了，来小乌鸦...,同桌如果讲喜趣的话呢？那要反过来讲。等一会下课再出去，啊？是不是讲得差不多了？来，小乌鸦来试...,"[\n {""type"": ""发起"", ""content"": ""同桌如果讲喜趣的话呢？那...","{\n ""result"": [\n {""type"": ""发起"", ""content""...","[\n {""type"": ""发起"", ""content"": ""同桌如果讲喜趣的话呢？那要反...","[\n {""type"":""讲解"",""content"":""同桌如果讲喜趣的话呢？那要反过来讲...","[\n {""type"": ""讲解"", ""content"": ""同桌如果讲喜趣的话呢？那要反...","\n[\n {""type"": ""发起"", ""content"": ""同桌如果讲喜趣的话呢？那..."
7,能够把故事讲用一个，第二个能加上动作，来来第三个能不能表演的更好，哎， 加上表情啊，你来试试...,能够把故事讲用一个。第二个能加上动作。来来，第三个能不能表演得更好？哎，加上表情啊。你来试试...,"[\n {""type"": ""发起"", ""content"": ""能够把故事讲得用一个。第...","{\n ""result"": [\n {\n\n""type"": ""发起"",\n\n""c...","[{""type"":""讲解"",""content"":""能够把故事讲用一个。第二个能加上动作。""}...","[\n {""type"":""发起"",""content"":""能够把故事讲用一个。""},\n ...","[\n {""type"": ""发起"", ""content"": ""能够把故事讲用一个。第二个能...","\n[\n {""type"": ""发起"", ""content"": ""能够把故事讲用一个。第二..."
8,不行，你按照老师的要求去做， 请坐，要批评， 你等会儿三十秒的话都到我这来，一定要积极思考，...,不行，你按照老师的要求去做。请坐。要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是...,"[\n{\n""type"":""评价"",\n""content"":""不行，你按照老师的要求去做。""...","{\n ""result"": [\n {\n\n""type"": ""其它"",\n\n""c...","[\n {""type"": ""发起"", ""content"": ""不行，你按照老师的要求去做。...","[\n {""type"":""其它"", ""content"":

In [69]:
df = pd.read_excel("data/qwen_72B_predict.xlsx")
df

,manual_content,label,content,predict
0,请袁艺喊“开始上课”，声音要响亮啊。,发起,请袁艺喊‘开始上课’，声音要响亮啊。,发起
1,真棒。,评价,真棒。,评价
2,好，现在用眼睛看，用心记住这个字，“猴子”。,发起,好，现在用眼睛看，用心记住这个字，‘猴子’。,讲解
3,它分别种了什么树呢？谁来说说？于凯，你来说说看。,发起,它分别种了什么树呢？谁来说说？,发起
4,它分别种了什么树呢？谁来说说？于凯，你来说说看。,发起,于凯，你来说说看。,发起
...,...,...,...,...
213,好，你来。,发起,好，你来。,发起
214,很好！,评价,很好！,评价
215,好，再来一个同学，李春月，你来。,发起,好，再来一个同学，李春月，你来。,发起
216,你说，“改革”。,发起,你说，“改革”。,评价


In [70]:
def highlight_errors(row):
    if row['label'] == '发起' and row['predict'] != '发起':
        # 召回计算中的错误，标记为黄色
        return ['background-color: yellow'] * len(row)
    elif row['predict'] == '发起' and row['label'] != '发起':
        # 精确率计算中的错误，标记为深绿色
        return ['background-color: darkgreen'] * len(row)
    else:
        return [''] * len(row)


In [71]:
df2 = df.style.apply(highlight_errors, axis=1)

In [73]:
df2.to_excel("data/qwen_72B_predict.xlsx", index=False)

In [1]:
text = {
        "start_time": [27300, 35310, 40560, 45590, 47910, 50070, 52780, 53000],
        "end_time": [32940, 39510, 42710, 47190, 49590, 52760, 52790, 69880],
        "text": [
            "具你，为什么要我买？这是第一套。",
            "喂，你，吃你吃你狗，你，",
            "好，把语文书翻到第50页，",
            "然后铅笔收起来把，",
            "课堂练习放到左上角，",
            "先把语文书翻到翻到第50页，翻到这里，",
            "没有，50。我现在这个阳猫世，",
            "我看谁今天的坐姿有问题啊啊，"
        ],
        "label": [1, 1, 1, 0, 1, 0, 1, 0]
    }

In [5]:
import pandas as pd 
pd.DataFrame(text)

,start_time,end_time,text,label
0,27300,32940,具你，为什么要我买？这是第一套。,1
1,35310,39510,喂，你，吃你吃你狗，你，,1
2,40560,42710,好，把语文书翻到第50页，,1
3,45590,47190,然后铅笔收起来把，,0
4,47910,49590,课堂练习放到左上角，,1
5,50070,52760,先把语文书翻到翻到第50页，翻到这里，,0
6,52780,52790,没有，50。我现在这个阳猫世，,1
7,53000,69880,我看谁今天的坐姿有问题啊啊，,0


In [27]:
test = {'eval_loss': 0.5066984295845032, 'eval_accuracy': 0.315644383184011, 'eval_f1_macro': 0.4497000921022335, 'eval_precision_macro': 0.5826596935086048, 'eval_recall_macro': 0.4690164720389716, 'eval_precision_class_0': 0.5596330275229358, 'eval_recall_class_0': 0.19003115264797507, 'eval_f1_class_0': 0.2837209302325581, 'eval_precision_class_1': 0.5294117647058824, 'eval_recall_class_1': 0.059602649006622516, 'eval_f1_class_1': 0.10714285714285714, 'eval_precision_class_2': 0.7299088649544325, 'eval_recall_class_2': 0.9342523860021209, 'eval_f1_class_2': 0.8195348837209302, 'eval_precision_class_3': 0.5116851168511685, 'eval_recall_class_3': 0.6921797004991681, 'eval_f1_class_3': 0.5884016973125884, 'eval_runtime': 274.9691, 'eval_samples_per_second': 5.277, 'eval_steps_per_second': 0.662, 'epoch': 16.0}


In [28]:
import pandas as pd 
df = pd.DataFrame(test,index=test.keys())

In [29]:
df

,eval_loss,eval_accuracy,eval_f1_macro,eval_precision_macro,eval_recall_macro,eval_precision_class_0,eval_recall_class_0,eval_f1_class_0,eval_precision_class_1,eval_recall_class_1,...,eval_precision_class_2,eval_recall_class_2,eval_f1_class_2,eval_precision_class_3,eval_recall_class_3,eval_f1_class_3,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
eval_loss,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_accuracy,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_f1_macro,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_precision_macro,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_recall_macro,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_precision_class_0,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_recall_class_0,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_f1_class_0,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_precision_class_1,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0
eval_recall_class_1,0.506698,0.315644,0.4497,0.58266,0.469016,0.559633,0.190031,0.283721,0.529412,0.059603,...,0.729909,0.934252,0.819535,0.511685,0.69218,0.588402,274.9691,5.277,0.662,16.0


In [30]:
df.to_excel("test.xlsx")

In [23]:
df = pd.read_excel("data/original_data/726.xlsx")

In [24]:
df

,start_time,end_time,text,label
0,6670.0,12580.0,是这个没有，你生病，,1
1,12880.0,18590.0,你帮妈去休息，每个，,1
2,19450.0,21780.0,呀都可以啊，每个人都可以，,1
3,26050.0,30750.0,自己该是可以学习的，就是如果是在，这方面，,1
4,35070.0,40350.0,定植物的牛奶吃巧克力，谢谢你们，,1
...,...,...,...,...
661,2383150.0,2385480.0,I py 洗，上下结构，,1
662,2385670.0,2388170.0,10字头一横二竖，,1
663,2388170.0,2390370.0,三横四竖五横折，,1
664,2390680.0,2392900.0,六横7.8撇点九，,1


In [26]:
{"start_time":df['start_time'].to_list(),"end_time":df['end_time'].to_list(),"text":df['text'].to_list(),'label':df['label'].to_list()}

{'start_time': [6670.0,
  12880.0,
  19450.0,
  26050.0,
  35070.0,
  40350.0,
  42190.0,
  53480.0,
  55540.0,
  58100.0,
  62980.0,
  67390.0,
  88190.0,
  91400.0,
  92880.0,
  94820.0,
  97040.0,
  98300.0,
  100020.0,
  102320.0,
  104760.0,
  106310.0,
  112090.0,
  113810.0,
  116770.0,
  118090.0,
  121310.0,
  122110.0,
  124550.0,
  127750.0,
  130670.0,
  135880.0,
  138280.0,
  141120.0,
  145830.0,
  149640.0,
  157640.0,
  158820.0,
  167780.0,
  175260.0,
  186880.0,
  188140.0,
  202820.0,
  207330.0,
  212730.0,
  215350.0,
  217730.0,
  220720.0,
  223020.0,
  226000.0,
  232280.0,
  234250.0,
  238210.0,
  240550.0,
  250810.0,
  258080.0,
  260290.0,
  261380.0,
  265710.0,
  270090.0,
  270470.0,
  273660.0,
  275480.0,
  278850.0,
  280310.0,
  284180.0,
  286960.0,
  288810.0,
  292830.0,
  295810.0,
  299640.0,
  303240.0,
  304160.0,
  307170.0,
  309490.0,
  310540.0,
  314450.0,
  318330.0,
  322650.0,
  324890.0,
  332870.0,
  337320.0,
  340060.0,
  343820.